<H2>課題4: 評判分析</H2>

<p>本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。</p>
<p>Amazon_reviewファイルにはTraining dataとTest dataが入っています。</p>
<p>Training dataを用いて機械学習を行い、その結果を元に6つのTest dataがpositiveかNegativeかを判別してください。</p>
<p>6章で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。</p>

<H2>1. 必要なモジュールの読み込み</H2>

In [1]:
import collections
import numpy as np
import numpy as np_amazon

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

import string
from string import punctuation

import glob
import os
import sys
import re

<H2>2. ディレクトリの確認</H2>

In [2]:
# カレントディレクトリの確認
print( os.listdir(os.path.normpath("./")) )

['.ipynb_checkpoints', 'Assignment1.ipynb', 'Assignment2.ipynb', 'Assignment3.ipynb', 'Assignment4.ipynb', 'dataset', 'mlbook-master', 'mlbook-master-new']


In [3]:
# データディレクトリの確認
print( os.listdir(os.path.normpath("./dataset/Amazon_review")) )

['Test_data', 'Training_data', '_DS_Store']


<H2>3. Dataの読み込み </H2>

フォルダ中のテキストへのパスを取得した上で各ファイルを読み込みます。

In [4]:
# default locale を ja_JP.UTF-8に設定する
def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


In [5]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Training set)
neg_files_training = glob.glob( os.path.normpath("./dataset/Amazon_review/Training_data/neg/*") )
pos_files_training = glob.glob( os.path.normpath("./dataset/Amazon_review/Training_data/pos/*") )

In [6]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(neg_files_training[0:2])
print(pos_files_training[0:2])

['dataset\\Amazon_review\\Training_data\\neg\\cv000_tok-9611.txt', 'dataset\\Amazon_review\\Training_data\\neg\\cv001_tok-19324.txt']
['dataset\\Amazon_review\\Training_data\\pos\\cv000_tok-11609.txt', 'dataset\\Amazon_review\\Training_data\\pos\\cv001_tok-10180.txt']


In [7]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Test set)
neg_files_test = glob.glob( os.path.normpath("./dataset/Amazon_review/Test_data/neg/*") )
pos_files_test = glob.glob( os.path.normpath("./dataset/Amazon_review/Test_data/pos/*") )

In [8]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(neg_files_test[0:2])
print(pos_files_test[0:2])

['dataset\\Amazon_review\\Test_data\\neg\\amazon_review_10000.txt', 'dataset\\Amazon_review\\Test_data\\neg\\amazon_review_10001.txt']
['dataset\\Amazon_review\\Test_data\\pos\\amaozn_review_20000.txt', 'dataset\\Amazon_review\\Test_data\\pos\\amaozn_review_20001.txt']


In [9]:
#  各ファイルを読み込む関数を定義
def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = re.sub( r'\*.\*.\*.\*.\*{0}+\*.\*.\*.\*{0}+\*.\*.\*{0}+\*.\*{0}+\*[^\.]+\.'.format('[^\*]'), "REVIEW_FORMAT" , line )
                line = re.sub(r'<a(.*?)</a>',"HTML_TAG", line)
                print(line)
    else:
        with open(file_path, 'r') as f:
            for line in f:
                line = re.sub( r'\*.\*.\*.\*.\*{0}+\*.\*.\*.\*{0}+\*.\*.\*{0}+\*.\*{0}+\*[^\.]+\.'.format('[^\*]'), "REVIEW_FORMAT" , line )
                line = re.sub(r'<a(.*?)</a>',"HTML_TAG", line)
                print(line)

In [10]:
# 読み込んだファイルの中身を表示（Test data, Negative, 1ファイル目）
text_reader(neg_files_test[0])

The Bad: Disregard for plot points from the lead up movies, including the first Avengers (mentioned, but stripped of all layers). Characters having sudden new and unexplained personalities (bound to happen when they're juggling so damned many!). CGI is a wonderful tool, but when you see Spider-Man's head floating above his CGI body (no he did not get decapitated), it pulls you out of the narrative. The focus of the film was on too many characters who were not relevant to the central plot, maybe the excuse was to have an epic Final Battle scene in Wakanda (SPOILER: Some of those dead characters are needed for a Black Panther sequel, further proving my point).



The Ugly: One character death after another, with so many dying that they often skip to the aftermath or just state that Thanos killed them, it really pulls at the heartstrings... Except, as this review title implies, the movie has zero lasting effects. They wasted no time in throwing out all credibility (SPOILER: Thor caused th

<H2> 4.特徴ベクトルの作成</H2>
<p>特徴ベクトルの作成に必要な関数を定義した後に特徴ベクトルを作成します。</p>
<p>特徴ベクトルはTraining setとTest setで同じ要素を持つ必要がありますので、両方のsetを合わせて作成し、後でTraining setとTest setに分けます。</p>

In [11]:
# 各単語の出現回数をカウントする関数を定義
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

In [12]:
# unigramを作成する関数を定義
def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf-8') as f:
                text = ''.join([line.rstrip('\r\n') for line in f])
                text = re.sub( r'\*.\*.\*.\*.\*{0}+\*.\*.\*.\*{0}+\*.\*.\*{0}+\*.\*{0}+\*[^\.]+\.'.format('[^\*]'), "REVIEW_FORMAT" , text )
                text = re.sub(r'<a(.*?)</a>',"HTML_TAG", text)
                count_dict = word_counter(text)
                result.append(count_dict)
    else:
        for file in file_path:
            with open(file, 'r') as f:
                text = ''.join([line.rstrip('\r\n') for line in f])
                text = re.sub( r'\*.\*.\*.\*.\*{0}+\*.\*.\*.\*{0}+\*.\*.\*{0}+\*.\*{0}+\*[^\.]+\.'.format('[^\*]'), "REVIEW_FORMAT" , text )
                text = re.sub(r'<a(.*?)</a>',"HTML_TAG", text)
                count_dict = word_counter(text)
                result.append(count_dict)
    

    return result

In [13]:
# Unigramをリストアップ (Training set + Test set)
unigrams_data = get_unigram(neg_files_training[:]) + get_unigram(pos_files_training[:]) \
                + get_unigram(neg_files_test[:]) + get_unigram(pos_files_test[:])

In [14]:
len(unigrams_data)

1406

In [15]:
# Unigramの内容を確認 (Training set)
print( unigrams_data[0] )

{'shirtless': 1, 'dennis': 3, 'level': 1, 'gratuitous': 1, "quinn's": 1, 'mass': 1, 'namely': 1, "who's": 1, 'frame': 1, 'be': 2, 'antwerp': 1, 'performance': 1, 'amusement': 1, 'the': 20, 'tries': 2, 'lite': 1, 'segment': 1, "neither's": 1, 'this': 2, 'set': 1, 'colony': 1, 'been': 1, 'tell': 1, 'soldiers': 1, 'than': 1, 'whatever': 1, 'ripe': 1, 'figure': 1, 'world': 1, '.': 21, 'them': 2, 'one': 2, 'story': 1, 'escapes': 1, 'high': 1, 'think': 1, 'jamie': 1, 'strange': 1, '?': 3, 'action': 1, 'are': 1, 'out': 3, 'aim': 1, 'juices': 1, 'r': 1, 'from': 1, 'weird-looking': 1, 'care': 1, 'expert': 1, 'exuberantly': 1, "he's": 3, 'damme': 4, 'on': 5, 'mones': 1, 'numerous': 1, 'make': 1, 'lindinger': 2, 'did': 2, 'locks': 1, 'gets': 1, 'arms': 1, 'cast': 1, 'stars': 1, 'ex-cia': 1, 'taken': 1, 'tiger': 1, "there's": 1, 'and': 17, 'frenetic': 1, 'else': 1, 'needs': 2, 'save': 1, "didn't": 1, 'known': 1, 'bad': 2, 'who': 1, 'basket': 1, 'seagal': 1, 'actually': 1, 'double': 5, 'movie-bedpo

In [16]:
# DictVectorizerを用いてUnigramを行列の形に変形　（各行が1つのレビュー、各列が単語、要素がその単語の出現数）
vec = DictVectorizer()
feature_vectors_csr = vec.fit_transform( unigrams_data )
feature_vectors = vec.fit_transform( unigrams_data ).toarray()

In [17]:
feature_vectors_csr

<1406x44266 sparse matrix of type '<class 'numpy.float64'>'
	with 492109 stored elements in Compressed Sparse Row format>

In [18]:
# 作成したデータを確認
print( "data dimension :", feature_vectors.shape )
print( feature_vectors[0] )
print( "data size :", sys.getsizeof(feature_vectors) / 1000000, "[MB]" )

data dimension : (1406, 44266)
[ 0.  0. 18. ...  0.  0.  0.]
data size : 497.90408 [MB]


## 5.ラベルデータの作成

今回扱うデータセットは全てに negative, positive というラベルが振られています。<br>
ここではそのラベルを negative → 0, positive → 1 とすることで二値判別問題のセットアップを構築します。<br>
ラベルも特徴ベクトルと同様にTraining setとTest setを合わせて作成し、後でTraining setとTest　setに分けます。

In [19]:
# ラベルデータの作成
labels = np.r_[np.tile(0, 700), np.tile(1, 700), np.tile(0, 3), np.tile(1, 3)]

In [20]:
#正しい位置で0と1の振替がなされているか確認 (Training setの0と1、Test setの0と1)
print( labels[0], labels[699], labels[700], labels[1399], \
      labels[1400], labels[1402], labels[1403], labels[1405]  )

0 0 1 1 0 0 1 1


<H2> 6. 学習用データと学習評価用データの作成</H2>
<p>今回はTree fold cross validationを行いますので、まずTraining setを3分割します。</p>

In [21]:
# 今回はThree fold cross validation でモデルを評価

In [22]:
# seedを固定し、乱数を1400個作成
np.random.seed(1010)
shuffle_order = np.random.choice( 1400, 1400, replace=False )

In [23]:
#生成した乱数の中身を確認
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [ 255 1231  979  696  760  238 1209  316 1008  605]


In [24]:
#作成した乱数を元にTraining setを3分割する
one_third_size = int( 1400 / 3. )
print( "one third of the length :", one_third_size )

print( "# of '1' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the length : 466
# of '1' in 1st set : 225
# of '1' in 2nd set : 235
# of '1' in 3rd set : 240


## 7. 学習に必要な関数の設定

1. 与えられたリストをN分割する関数<br>

In [25]:
# 与えられたリストをN分割する関数を定義
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)] )
        last += avg
        
    return np.array(out)

In [26]:
# 動作を確認
N_splitter(range(11), 3)

array([range(0, 3), range(3, 7), range(7, 11)], dtype=object)

<p>2. train_model : 説明変数とラベルと手法を与えることでモデルを学習する</p>

In [27]:
# train_model 
def train_model(features, labels, method='SVM', parameters=None):
    # モデル指定
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    # パラメータがあれば指定
    if parameters:
        model.set_params(**parameters)
    # モデルを学習させる
    model.fit( features, labels )
    
    return model

<p>3. predict : モデルと説明変数を与えることでラベルを予測する</p>

In [28]:
# predict 
def predict(model, features):
    predictions = model.predict( features )
    return predictions

<p>4. evaluate_model : 予測したラベルと実際の答えの合致数を調べる

In [29]:
#evaluate_model 
def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

<p> 5. cross_validate : cross_validationを実行する </p>

In [30]:
# cross_validate
def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(1400), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # 学習
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # 予測
        predictions = predict( model, feature_vectors[test_elems] )
        # 評価
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num ,model

## 8. 学習の実施、精度の検証

In [31]:
%%time
# Trainingを実施
ans,corr,model = cross_validate(3, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=None)

Executing 1th set...
Executing 2th set...
Executing 3th set...
Wall time: 11.9 s


In [32]:
# 結果を表示
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  63.9 %


## 9. パラメータのチューニング

In [33]:
%%time
#パラメータをチューニング（どのような形でも構いません）
feature_vectors_csr.data[ feature_vectors_csr.data > 0 ] = 1.

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

Wall time: 6min 55s


In [34]:
# 結果を表示
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)

best paremters :  {'gamma': 0.001, 'C': 10, 'kernel': 'rbf'}
best scores :  0.8193456614509246


In [35]:
%%time
# チューニング後のパラメータで学習
ans,corr,model = cross_validate(3, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...
Wall time: 11.7 s


In [36]:
# 結果を表示
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  81.7 %


## 10. Test dataを用いてモデルを評価

In [37]:
# 上記で作成したモデルを用いてTest dataの評価を実施
ans,corr = evaluate_model( predict( model, feature_vectors_csr[1400:] ), labels[1400:] )

In [38]:
# 結果を表示
print( "average precision : ", np.around( 100.*corr/ans, decimals=1 ), "%" )

average precision :  66.7 %
